# Import PyTorch modules

In [1]:
import torch
import torchvision
from torchvision.datasets import VOCDetection
import os

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", DEVICE)

Device: cpu


# Load datasets

In [3]:
import os

from pathlib import Path

from google.colab import drive

drive.mount('/content/drive')

SHARED_PATH = Path("drive/MyDrive/Colab Notebooks/Shared")
HRSC_PATH = SHARED_PATH / "HRSC2016_Final_Splits"
DOTA_PATH = SHARED_PATH / "DOTA_Final_Splits"

print("HRSC subfolders:", [f"{subfolder.name}/{path.name}" for subfolder in HRSC_PATH.iterdir() for path in subfolder.iterdir()])
print("DOTA subfolders:", [f"{subfolder.name}/{path.name}" for subfolder in DOTA_PATH.iterdir() for path in subfolder.iterdir()])

HRSC_TRAIN_IMAGES = HRSC_PATH/ "train/images"
HRSC_TRAIN_ANNOTATIONS = HRSC_PATH / "train/annotations"
HRSC_VAL_IMAGES = HRSC_PATH / "val/images"
HRSC_VAL_ANNOTATIONS = HRSC_PATH / "val/annotations"
HRSC_TEST_IMAGES = HRSC_PATH / "test/images"
HRSC_TEST_ANNOTATIONS = HRSC_PATH / "test/annotations"

DOTA_TRAIN_IMAGES = DOTA_PATH / "train/images"
DOTA_TRAIN_ANNOTATIONS = DOTA_PATH / "train/hbb"
DOTA_VAL_IMAGES = DOTA_PATH / "val/images"
DOTA_VAL_ANNOTATIONS = DOTA_PATH / "val/hbb"
DOTA_TEST_IMAGES = DOTA_PATH / "test/images"
DOTA_TEST_ANNOTATIONS = DOTA_PATH / "test/hbb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
HRSC subfolders: ['train/images', 'train/annotations', 'val/images', 'val/annotations', 'test/images', 'test/annotations']
DOTA subfolders: ['train/images', 'train/hbb', 'val/images', 'val/hbb', 'test/images_without_hbb', 'test/images', 'test/hbb']


# Explore datasets

In [4]:
!pip install colorama
from colorama import Fore, Style

def explore_header(dataset: str, subfolder: str):
  print("Exploring", Fore.GREEN + dataset + Style.RESET_ALL, Fore.MAGENTA + subfolder + Style.RESET_ALL, "folder...")

def explore(images_folder: Path, ext: str):
  files = list(images_folder.glob(f"*.{ext}"))
  print(f"Number of {ext.upper()} files:", len(files))
  print(f"{ext.upper()} sample files:", [path.name for path in files[:3]])

explore_header("HRSC", "train")
explore(HRSC_TRAIN_IMAGES, "bmp")
explore(HRSC_TRAIN_ANNOTATIONS, "xml")

print()

explore_header("HRSC", "val")
explore(HRSC_VAL_IMAGES, "bmp")
explore(HRSC_VAL_ANNOTATIONS, "xml")

print()

explore_header("HRSC", "test")
explore(HRSC_TEST_IMAGES, "bmp")
explore(HRSC_TEST_ANNOTATIONS, "xml")

print()

explore_header("DOTA", "train")
explore(DOTA_TRAIN_IMAGES, "png")
explore(DOTA_TRAIN_ANNOTATIONS, "txt")

print()

explore_header("DOTA", "val")
explore(DOTA_VAL_IMAGES, "png")
explore(DOTA_VAL_ANNOTATIONS, "txt")

print()

explore_header("DOTA", "test")
explore(DOTA_TEST_IMAGES, "png")
explore(DOTA_TEST_ANNOTATIONS, "txt")

Exploring HRSC train folder...
Number of BMP files: 436
BMP sample files: ['100000001.bmp', '100000002.bmp', '100000004.bmp']
Number of XML files: 436
XML sample files: ['100000001.xml', '100000002.xml', '100000004.xml']

Exploring HRSC val folder...
Number of BMP files: 181
BMP sample files: ['100000006.bmp', '100000010.bmp', '100000622.bmp']
Number of XML files: 181
XML sample files: ['100000006.xml', '100000010.xml', '100000622.xml']

Exploring HRSC test folder...
Number of BMP files: 453
BMP sample files: ['100000003.bmp', '100000005.bmp', '100000623.bmp']
Number of XML files: 453
XML sample files: ['100000003.xml', '100000005.xml', '100000623.xml']

Exploring DOTA train folder...
Number of PNG files: 941
PNG sample files: ['P0000.png', 'P0001.png', 'P0020.png']
Number of TXT files: 941
TXT sample files: ['P0038.txt', 'P0039.txt', 'P0041.txt']

Exploring DOTA val folder...
Number of PNG files: 458
PNG sample files: ['P0003.png', 'P0004.png', 'P0007.png']
Number of TXT files: 458
TX

# Define label parsers

In [5]:
import itertools
import xml.etree.ElementTree as ET

HRSC_CLASSES = {}
DOTA_CLASSES = {}


def parse_hrsc_labels(label_path: Path):
  boxes = []
  labels = []
  tree = ET.parse(label_path.as_posix())
  root = tree.getroot()

  # Get image dimensions
  width = int(root.find('.//Img_SizeWidth').text)
  height = int(root.find('.//Img_SizeHeight').text)

  objects = root.findall(".//HRSC_Object")
  for obj in objects:
    try:
      # Bounds
      xmin = float(obj.find('box_xmin').text)
      ymin = float(obj.find('box_ymin').text)
      xmax = float(obj.find('box_xmax').text)
      ymax = float(obj.find('box_ymax').text)

      # Category
      class_id = int(obj.find('Class_ID').text)
      if class_id not in HRSC_CLASSES:
        HRSC_CLASSES[class_id] = len(HRSC_CLASSES)
      class_id_index = HRSC_CLASSES[class_id]

      boxes.append([xmin, ymin, xmax, ymax])
      labels.append(class_id_index)

    except Exception as e:
      print(Fore.RED + "Warning" + Style.RESET_ALL + f": Could not parse object in {label_path.as_posix()}: {e}")
      continue

  return boxes, labels


def parse_dota_labels(label_path: Path):
  boxes = []
  labels = []

  # Find all objects
  for line in itertools.islice(label_path.read_text().splitlines(), 2, None):  # Start from line index 2
    try:
      obj = line.strip().split()
      x1, y1, x2, y2, x3, y3, x4, y4, category, difficulty = (*map(float, obj[:8]), *obj[8:])

      # Bounds
      xmin = min(x1, x2, x3, x4)
      ymin = min(y1, y2, y3, y4)
      xmax = max(x1, x2, x3, x4)
      ymax = max(y1, y2, y3, y4)

      # Category
      if category not in DOTA_CLASSES:
        DOTA_CLASSES[category] = len(DOTA_CLASSES)
      class_id_index = DOTA_CLASSES[category]

      boxes.append([xmin, ymin, xmax, ymax])
      labels.append(class_id_index)

    except Exception as e:
      print(Fore.RED + "Warning" + Style.RESET_ALL + f": Could not parse object in {label_path.as_posix()}: {e}")
      continue

  return boxes, labels

# PyTorch dataset structure

In [7]:
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import torchvision.transforms as T

from PIL import Image

BASIC_TRANSFORM = T.Compose([T.ToTensor()])

class TorchDataset(torch.utils.data.Dataset):
  def __init__(self, images_folder: Path, annotations_folder: Path, label_parser, transforms=BASIC_TRANSFORM) -> None:
    super().__init__()
    self.label_parser = label_parser
    self.transforms = transforms
    images = [f for f in images_folder.iterdir() if f.suffix in (".jpg", ".png", ".bmp")] if images_folder.exists() else []
    annotations = [f for f in annotations_folder.iterdir() if f.suffix in (".xml", ".txt")] if annotations_folder.exists() else []

    # Keep only images/annotations one-to-one correspondences
    image_set = set(f.stem for f in images)
    annotation_set = set(f.stem for f in annotations)
    self.ids = image_set.intersection(annotation_set)
    self.images = {f.stem: f for f in images if f.stem in self.ids}
    self.annotations = {f.stem: f for f in annotations if f.stem in self.ids}
    self.ids = list(self.ids)

  def __getitem__(self, index):
    id = self.ids[index]
    image_path = self.images[id]
    label_path = self.annotations[id]
    image = Image.open(image_path).convert("RGB")
    boxes, labels = self.label_parser(label_path)
    target = {
        "boxes": torch.as_tensor(boxes, dtype=torch.float32),
        "labels": torch.as_tensor(labels, dtype=torch.int64),
    }
    image = self.transforms(image)
    return image, target

  def __len__(self):
    return len(self.ids)

  def compute_total_number_of_objects(self, max_workers=8):
    def count_objects(id):
      boxes, _ = self.label_parser(self.annotations[id])
      return len(boxes)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
      results = executor.map(count_objects, self.ids)
    return sum(results)

# Prepare datasets for PyTorch

In [8]:
print("Preparing HRSC training dataset...")
HRSC_TRAIN_DATASET = TorchDataset(HRSC_TRAIN_IMAGES, HRSC_TRAIN_ANNOTATIONS, parse_hrsc_labels)
print(f"...Dataset prepared: {HRSC_TRAIN_DATASET.compute_total_number_of_objects()} total objects")
print("HRSC training dataset sample:", HRSC_TRAIN_DATASET.ids[:5])

print("Preparing HRSC validation dataset...")
HRSC_VAL_DATASET = TorchDataset(HRSC_VAL_IMAGES, HRSC_VAL_ANNOTATIONS, parse_hrsc_labels)
print(f"...Dataset prepared: {HRSC_VAL_DATASET.compute_total_number_of_objects()} total objects")
print("HRSC validation dataset sample:", HRSC_VAL_DATASET.ids[:5])

print("Preparing HRSC testing dataset...")
HRSC_TEST_DATASET = TorchDataset(HRSC_TEST_IMAGES, HRSC_TEST_ANNOTATIONS, parse_hrsc_labels)
print(f"...Dataset prepared: {HRSC_TEST_DATASET.compute_total_number_of_objects()} total objects")
print("HRSC testing dataset sample:", HRSC_TEST_DATASET.ids[:5])

print("Preparing DOTA training dataset...")
DOTA_TRAIN_DATASET = TorchDataset(DOTA_TRAIN_IMAGES, DOTA_TRAIN_ANNOTATIONS, parse_dota_labels)
print(f"...Dataset prepared: {DOTA_TRAIN_DATASET.compute_total_number_of_objects()} total objects")
print("DOTA training dataset sample:", DOTA_TRAIN_DATASET.ids[:5])

print("Preparing DOTA validation dataset...")
DOTA_VAL_DATASET = TorchDataset(DOTA_VAL_IMAGES, DOTA_VAL_ANNOTATIONS, parse_dota_labels)
print(f"...Dataset prepared: {DOTA_VAL_DATASET.compute_total_number_of_objects()} total objects")
print("DOTA validation dataset sample:", DOTA_VAL_DATASET.ids[:5])

print("Preparing DOTA testing dataset...")
DOTA_TEST_DATASET = TorchDataset(DOTA_TEST_IMAGES, DOTA_TEST_ANNOTATIONS, parse_dota_labels)
print(f"...Dataset prepared: {DOTA_TEST_DATASET.compute_total_number_of_objects()} total objects")
print("DOTA testing dataset sample:", DOTA_TEST_DATASET.ids[:5])

Preparing HRSC training dataset...
...Dataset prepared: 1207 total objects
HRSC training dataset sample: ['100001571', '100000982', '100001634', '100001320', '100000821']
Preparing HRSC validation dataset...
...Dataset prepared: 541 total objects
HRSC validation dataset sample: ['100001336', '100001479', '100000006', '100000641', '100001679']
Preparing HRSC testing dataset...
...Dataset prepared: 1228 total objects
HRSC testing dataset sample: ['100000716', '100001491', '100001384', '100000659', '100001517']
Preparing DOTA training dataset...
...Dataset prepared: 143302 total objects
DOTA training dataset sample: ['P1070', 'P0522', 'P1842', 'P0821', 'P0580']
Preparing DOTA validation dataset...
...Dataset prepared: 69565 total objects
DOTA validation dataset sample: ['P0684', 'P1005', 'P0198', 'P1398', 'P0837']
Preparing DOTA testing dataset...
...Dataset prepared: 67329 total objects
DOTA testing dataset sample: ['P1084', 'P1980', 'P0677', 'P0535', 'P2783']


# Define PyTorch Faster R-CNN model

In [9]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms as T

class FasterRCNNModel:
  def __init__(self, train_dataset: TorchDataset, val_dataset: TorchDataset, test_dataset: TorchDataset, class_names: list, batch_size=4, shuffle_datasets=False) -> None:
    self.train_dataset = train_dataset
    self.val_dataset = val_dataset
    self.test_dataset = test_dataset

    self.class_names = list(class_names) + ["__background__"]
    self.num_classes = len(class_names)

    collate_fn = lambda x: tuple(zip(*x))
    self.train_loader = torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size, shuffle=shuffle_datasets, collate_fn=collate_fn)
    self.val_loader = torch.utils.data.DataLoader(self.val_dataset, batch_size=batch_size, shuffle=shuffle_datasets, collate_fn=collate_fn)
    if len(self.test_dataset) > 0:
        self.test_loader = torch.utils.data.DataLoader(self.test_dataset, batch_size=batch_size, shuffle=shuffle_datasets, collate_fn=collate_fn)
    else:
        self.test_loader = None

    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    self.model = fasterrcnn_resnet50_fpn(weights=weights)
    self.in_features = self.model.roi_heads.box_predictor.cls_score.in_features
    self.model.roi_heads.box_predictor = FastRCNNPredictor(self.in_features, self.num_classes)
    self.model.to(DEVICE)

  def train(self, num_epochs=50):
    # TODO Add an "RoI Learner" after ROI pooling to predit rotated offsets (dx, dy, dw, dh, dtheta)
    # TODO Replace roi_align with torchvision.ops.roi_align_rotated
    # TODO Add rotation-aware losses

    # Example optimizer
    optimizer = torch.optim.SGD(self.model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
      print(f"Starting epoch {epoch+1}/{num_epochs}...")

      # Training loop
      print("\tStarting training loop...")
      train_loss = 0.0
      self.model.train()
      for images, targets in self.train_loader:
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = self.model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss += losses.item()

      lr_scheduler.step()
      print("\t...Training loop complete.")

      # Validation loop
      print("\tStarting validation loop...")
      val_loss = 0.0
      self.model.eval()
      with torch.no_grad():
        for images, targets in self.val_loader:
          images = [img.to(DEVICE) for img in images]
          targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
          loss_dict = self.model(images, targets)
          losses = sum(loss for loss in loss_dict.values())
          val_loss += losses.item()
      print("\t...Validation loop complete.")

      print(f"...Finished epoch {epoch+1}/{num_epochs}, Training loss: {train_loss:.4f}, Validation loss: {val_loss:.4f}")

    torch.save({
        "model_state_dict": self.model.state_dict(),
        "class_names": self.class_names
    }, "faster_rcnn_model.pth")

  def test_results(self):
    if self.test_loader is None:
        print("Test dataset is empty. Skipping evaluation.")
        return [], [], []

    self.model.eval()
    all_boxes, all_labels, all_scores = [], [], []
    with torch.no_grad():
      for images, _ in self.test_loader:
        images = [img.to(DEVICE) for img in images]
        predictions = self.model(images)
        for prediction in predictions:
          boxes, labels, scores = prediction['boxes'], prediction['labels'], prediction['scores']
          all_boxes.append(boxes.cpu().numpy())
          all_labels.append(labels.cpu().numpy())
          all_scores.append(scores.cpu().numpy())
    return all_boxes, all_labels, all_scores

# TODO
# -------------------------------------------------------------------
# FUTURE UPGRADE: RoI TRANSFORMER INTEGRATION
# -------------------------------------------------------------------
# To integrate the RoI Transformer:
#   1. Add a custom RRoI Learner layer:
#        fc = nn.Linear(roi_feature_dim, 5)
#        -> predicts (dx, dy, dw, dh, dtheta)
#   2. Compute rotated boxes and apply torchvision.ops.roi_align_rotated
#   3. Replace standard SmoothL1 loss with rotated IoU or 5D regression loss
#   4. Use torchvision.ops.box_iou_rotated() and nms_rotated() for matching and inference
#   5. Dataset boxes should include rotation (x, y, w, h, theta)

# Prepare dataset models

In [10]:
HRSC_MODEL = FasterRCNNModel(HRSC_TRAIN_DATASET, HRSC_VAL_DATASET, HRSC_TEST_DATASET, [id for id, _ in sorted(HRSC_CLASSES.items(), key=lambda item: item[1])])
DOTA_MODEL = FasterRCNNModel(DOTA_TRAIN_DATASET, DOTA_VAL_DATASET, DOTA_TEST_DATASET, [id for id, _ in sorted(DOTA_CLASSES.items(), key=lambda item: item[1])])

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:00<00:00, 263MB/s]


# Train dataset models

In [ ]:
print("Training HRSC model...")
HRSC_MODEL.train()
print("...HRSC model trained.")

print("Training DOTA model...")
DOTA_MODEL.train()
print("...DOTA model trained.")

Training HRSC model...
Starting epoch 1/50...
	Starting training loop...


# Evaluate dataset models

In [ ]:
hrsc_boxes, hrsc_labels, hrsc_scores = HRSC_MODEL.test_results()
dota_boxes, dota_labels, dota_scores = DOTA_MODEL.test_results()
# TODO analyse results and visualize them